# NLP Homework 8
## Sentiment Analysis
### Julia Layne

In [2]:
import pandas as pd
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn
from nltk.corpus import sentiwordnet as swn
from nltk import sent_tokenize, word_tokenize, pos_tag
import re

## Perform a vocabulary-based sentiment analysis of the movie reviews

## Data Sources
This dataset consists of reviews pulled from IMDB. It contains Comedy Genre TV Shows and Movies . In previous work, the permalinks for these reviews were used to pull in all HTML of these pages and the review paragraphs pulled out via BeautifulSoup.  All non alphanumeric characters were stripped and replaced with spaces. In this notebook, these text paragraphs were tokenized and the tagged for parts of speech.

## Technique
 To get the sentiment of these reviews, I used the package SentiWordNet. I took the content of the review and the parts of speech tags and modified them to a version that SentiWordNet understood in `penn_to_wn`. The first sentiment of the word was taken and the positive and negative were subtracted from one another to get the overall sentiment. This was done for each word and summed up for each word. 

## Findings

Based on this analysis, when comparing to my original thoughts reviewing, the sentiment analysis really reflects the mix of reviews within clusters. There weren't as clear dividing lines among reviews, however I found that this matched the 'sarcasm' of the reviews.

When picking television and comedy, I started pulling from the top 100 comedies and found that these had many negative reviews in addition to the mostly positive feedback. However, because of the content I think Cluster 0 in particular is getting analyzed incorrectly. They have words like 'laugh','funny', and more comedy expectations reviewers are saying the show did not meet. There are also the jeering nature of some of the reviews: 'most laughingly awful', 'Amazing---ly Awful'.

The dark comedy (I Blame Society) was picked before I went to the top comedies list. It makes it somewhat different in that it generally has fewer reviews, but also has horror aspect to it. Even positive reviews tended to get linked to negative reviews. A very high review of this movie was placed with some more negative reviews in Cluster 0 and definitely skewed positive with a sentiment score of 11.25. 

In [3]:
reviews = pd.read_csv("parsed_reviews_sentiment.csv")
for i in range(6):
    print('---- Cluster ',i,'----')
    filtered = reviews['cluster']==i
    review_filtered = reviews.copy()
    review_filtered.where(filtered, inplace = True)
    review_filtered = review_filtered.dropna()
    #average, median, high, and low
    print('Average: ',review_filtered['SENTIMENT'].mean())
    print('Median: ',review_filtered['SENTIMENT'].median())
    print('High: ',review_filtered['SENTIMENT'].max())
    print('Low: ',review_filtered['SENTIMENT'].min())
    if review_filtered['SENTIMENT'].median() >0:
        print('Overall Positive')
    else:
        print('Overall Negative')
    print("Old Description:\n",oldDescription[i])
    

---- Cluster  0 ----
Average:  3.2083333333333335
Median:  2.3125
High:  11.25
Low:  -0.125
Overall Positive
Old Description:
 Mostly negative MidSized reviews. One postiive for a dark comedy
---- Cluster  1 ----
Average:  0.49375
Median:  0.5
High:  5.75
Low:  -3.5
Overall Positive
Old Description:
 Some positive and Luke-warm reviews
---- Cluster  2 ----
Average:  0.90625
Median:  0.1875
High:  4.0
Low:  -0.75
Overall Positive
Old Description:
 Mostly Positive Short reviews
---- Cluster  3 ----
Average:  1.2022068965517243
Median:  1.25
High:  8.25
Low:  -6.375
Overall Positive
Old Description:
 Mostly Positive, even the negative reviews are softer: 'The premise is cute and the movie seemed like it was going places'
---- Cluster  4 ----
Average:  0.25
Median:  -0.0625
High:  5.0
Low:  -3.5
Overall Negative
Old Description:
 Mostly lukewarm reviews
---- Cluster  5 ----
Average:  -1.375
Median:  -1.375
High:  -0.375
Low:  -2.375
Overall Negative
Old Description:
 Mostly Negative review

# Code

### Submit all of your inputs and outputs and your code for this assignment, along with a brief written explanation of your findings.

### 1.	In Python, load one of the sentiment vocabularies referenced in the textbook, and run the sentiment analyzer as explained in the corresponding reference. Add words to the sentiment vocabulary, if you think you need to, to better fit your particular text collection.

In [4]:
reviews = pd.read_csv("parsed_reviews_clustered6.csv")
reviews["SENTIMENT"] = '-'

In [5]:
#Sourced from this article with some changes to get the swap from pennbank 
#https://nlpforhackers.io/sentiment-analysis-intro/

lemmatizer = WordNetLemmatizer()

def penn_to_wn(tag):
    if tag.startswith('J'):
        return wn.ADJ
    if tag.startswith('N'):
        return wn.NOUN
    if tag.startswith('R'):
        return wn.ADV
    if tag.startswith('V'):
        return wn.VERB
    return None
 
def clean_text(text):
    text = re.sub(r'[^\w\s]',' ',text)
    return text
 
def swn_polarity(text):
    sentiment = 0.0
    tokens_count = 0
    text = clean_text(text)
    raw_sentences = sent_tokenize(text)
    for raw_sentence in raw_sentences:
        tagged_sentence = pos_tag(word_tokenize(raw_sentence))
 
        for word, tag in tagged_sentence:
            wn_tag = penn_to_wn(tag)
            if wn_tag not in (wn.NOUN, wn.ADJ, wn.ADV):
                continue
            lemma = lemmatizer.lemmatize(word, pos=wn_tag)
            if not lemma:
                continue
            synsets = wn.synsets(lemma, pos=wn_tag)
            if not synsets:
                continue
            # Take the first sense, the most common
            synset = synsets[0]
            swn_synset = swn.senti_synset(synset.name())
            sentiment += swn_synset.pos_score() - swn_synset.neg_score()
            tokens_count += 1
    #return the overall sentiment instead of the 0/1 for positive negative to allow for handling later
    return sentiment

In [6]:
# Since we're shuffling, you'll get diffrent results
print(swn_polarity(reviews["CONTENT"][0]))

-0.75


In [7]:
for i in reviews.index:
    text = reviews["CONTENT"][i]
    sentiment = swn_polarity(text)
    #print(sentiment)
    reviews["SENTIMENT"][i] = sentiment

<ipython-input-7-b93bb20e8b2d>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews["SENTIMENT"][i] = sentiment


In [8]:
reviews.to_csv(r'parsed_reviews_sentiment.csv', index = False)

In [9]:
reviews.head()

URL  \
0  https://www.imdb.com/review/rw6540019/?ref_=tt...   
1  https://www.imdb.com/review/rw6562703/?ref_=tt...   
2  https://www.imdb.com/review/rw6650913/?ref_=tt...   
3  https://www.imdb.com/review/rw6603814/?ref_=tt...   
4  https://www.imdb.com/review/rw6479388/?ref_=tt...   

                                                SOUP  \
0  \r\n<!DOCTYPE html>\r\n\r\n<html xmlns:fb="htt...   
1  \r\n<!DOCTYPE html>\r\n\r\n<html xmlns:fb="htt...   
2  \r\n<!DOCTYPE html>\r\n\r\n<html xmlns:fb="htt...   
3  \r\n<!DOCTYPE html>\r\n\r\n<html xmlns:fb="htt...   
4  \r\n<!DOCTYPE html>\r\n\r\n<html xmlns:fb="htt...   

                                             CONTENT  \
0  It might have been an error on Disney's part t...   
1  Writing this with bated breath waiting on epis...   
2  One might wonder that the above adjectives do ...   
3  I can't speak for the reviews that bash this s...   
4  What a waste $25 Million Dollars A episode don...   

                                            NP_CHUNK  cluster SENTIMENT  
0  ['It', 'an', 'error', 'Disney', 'part', 'episo...        2     -0.75  
1  ['bated', 'breath', 'episode', 'The', 'whole',...        0         2  
2  ['the', 'above', 'adjectives', 'the', 'Marvel'...        1      -3.5  
3  ['I', 'the', 'reviews', 'this', 'show', 'three...        0     2.625  
4  ['a', 'waste', '25', 'Million', 'Dollars', 'A'...        0    -0.125

### 2.	 For each of the clusters you created in homework 7, compute the average, median, high, and low sentiment scores for each cluster. Explain whether you think this reveals anything interesting about the clusters.

In [10]:
oldDescription = [
    "Mostly negative MidSized reviews. One postiive for a dark comedy",
    "Some positive and Luke-warm reviews",
    "Mostly Positive Short reviews",
    "Mostly Positive, even the negative reviews are softer: 'The premise is cute and the movie seemed like it was going places'",
    "Mostly lukewarm reviews",
    "Mostly Negative reviews of popular shows"
]

In [11]:
for i in range(6):
    print('---- Cluster ',i,'----')
    filtered = reviews['cluster']==i
    review_filtered = reviews.copy()
    review_filtered.where(filtered, inplace = True)
    review_filtered = review_filtered.dropna()
    #average, median, high, and low
    print('Average: ',review_filtered['SENTIMENT'].mean())
    print('Median: ',review_filtered['SENTIMENT'].median())
    print('High: ',review_filtered['SENTIMENT'].max())
    print('Low: ',review_filtered['SENTIMENT'].min())
    if review_filtered['SENTIMENT'].median() >0:
        print('Overall Positive')
    else:
        print('Overall Negative')
    print("Old Description:\n",oldDescription[i])
    


---- Cluster  0 ----
Average:  3.2083333333333335
Median:  2.3125
High:  11.25
Low:  -0.125
Overall Positive
Old Description:
 Mostly negative MidSized reviews. One postiive for a dark comedy
---- Cluster  1 ----
Average:  0.49375
Median:  0.5
High:  5.75
Low:  -3.5
Overall Positive
Old Description:
 Some positive and Luke-warm reviews
---- Cluster  2 ----
Average:  0.90625
Median:  0.1875
High:  4.0
Low:  -0.75
Overall Positive
Old Description:
 Mostly Positive Short reviews
---- Cluster  3 ----
Average:  1.2022068965517243
Median:  1.25
High:  8.25
Low:  -6.375
Overall Positive
Old Description:
 Mostly Positive, even the negative reviews are softer: 'The premise is cute and the movie seemed like it was going places'
---- Cluster  4 ----
Average:  0.25
Median:  -0.0625
High:  5.0
Low:  -3.5
Overall Negative
Old Description:
 Mostly lukewarm reviews
---- Cluster  5 ----
Average:  -1.375
Median:  -1.375
High:  -0.375
Low:  -2.375
Overall Negative
Old Description:
 Mostly Negative review

### 3.	For extra credit, analyze sentiment of chunks as follows:
 - a.	Take the chunks from homework 5, and in Python, run each chunk individually through your sentiment analyzer that you used in question 1. If the chunk registers a nonneutral sentiment, save it in a tabular format (the chunk, the sentiment score).
 - b.	Now sort the table twice, once to show the highest negative-sentiment-scoring chunks at the top and again to show the highest positive-sentiment-scoring chunks at the top. Examine the upper portions of both sorted lists, to identify any trends, and explain what you see. 
